### GOAL

Simulate data 
Related issue: https://gitlab.com/gophd/ecdna-simulation-classification/-/issues/1

### TODO
- develop ecDNA simulation workflow
- simulate simple excisions
- simulate simple SV (small DEL, INV)
- simulate intra-chromosomal multi-region including INV
- simulate inter-chromosomal multi-region including INV
- simulate all above allowing segmental duplication


In [96]:
import pandas as pd
from scipy.stats import nbinom
import random

chrlen = {"chr2":242193529,
         "chr12":133275309}

chrranges = {"chr2":(4880899,83143574),
             "chr12":(48581298,120529644)}

# n = number of fragments
# single_chr = intrachromosomal (True), interchromosomal (False)

N = "n"
NEIGHBOR = "neighbor"
SINGLE_CHR = "single_chr"
WEIGHT_CHR = "weights_chr"
P_DEL_LEFT = "p_del_left"
P_DEL_RIGHT = "p_del_right"
P_INVERT = "p_invert"
P_DUP = "p_dup"

conf = {N:3,
        NEIGHBOR:False,
        SINGLE_CHR:True,
        WEIGHT_CHR: [50,50],
        P_DEL_LEFT: 20,
        P_DEL_RIGHT: 30,
        P_INVERT: 40,
        P_DUP: 10
       }

def pick_chr(chrlen, conf):
    """
    Choose ecDNA origin chromosome
    """
    return random.choices([p for p in chrlen], weights=conf[WEIGHT_CHR], k=1)[0]

def pick_pos(chrranges, conf, chr_):
    """
    Pick genomic position on chr
    """
    return random.choices(range(chrranges[chr_][0], chrranges[chr_][1]), k=1)[0]
    
def pick_len(chrlen, conf, chr_, pos_, meansize = 20000, sd=5000):
    """
    Pick length of ecDNA fragment
    """
    return random.choices(range(meansize-sd, meansize+sd, 100), k=1)[0]

def choose_fragment_random(chrlen, chrranges, conf):
    """
    Choose genomic fragment of origin
    """
    chr_ = pick_chr(chrlen, conf)
    start_ = pick_pos(chrranges, conf, chr_)
    len_ = pick_len(chrlen, conf, chr_, start_)
    end_ = start_ + len_
    
    return chr_, start_, end_, len_

def choose_fragment_neighbor(chrlen, chrranges, conf, index, ecDNA):
    """
    Choose genomic fragment of origin
    """
    if index == -1:
        chr_ = pick_chr(chrlen, conf)
        start_ = pick_pos(chrranges, conf, chr_)
        len_ = pick_len(chrlen, conf, chr_, start_)
        end_ = start_ + len_
    else:
        # 0 1 2 3 4 5
        # chr_,start_,end_,strand_,count_frag
        chr_ = ecDNA[index][0] 
        start_ = ecDNA[index][2] 
        len_ = pick_len(chrlen, conf, chr_, start_)
        end_ = start_ + len_
    
    return chr_, start_, end_, len_

def choose_fragment(chrlen, chrranges, index, ecDNA, conf, type="random"):
    
    chr_, start_, end_, len_ = None, None, None, None
    if conf[NEIGHBOR] == True:
        chr_, start_, end_, len_ =  choose_fragment_neighbor(chrlen, chrranges, conf, index, ecDNA)
    else:
        chr_, start_, end_, len_ =  choose_fragment_random(chrlen, chrranges, conf)
    return   chr_, start_, end_, len_  
    

def crop_left_end(fragment_len, p):
    """
    Decide to crop (10% of fragment len) or not the left end 
    """
    del_left = random.choices([1,0], weights=[p,100-p], k=1)[0]
    
    return (del_left, 0.1*fragment_len)

def crop_right_end(fragment_len, p):
    """
    Decide to crop (10% of fragment len) or not the right end 
    """
    del_right = random.choices([1,0], weights=[p,100-p], k=1)[0]
    
    return (del_right, 0.1*fragment_len)

def small_deletions(chrlen, chr_, start_, end_, len_, conf):
    """
    Emulate small deletions
    """
    
    (left, lsize) = crop_left_end(len_, conf[P_DEL_LEFT])
    (right, rsize) = crop_right_end(len_, conf[P_DEL_RIGHT])
    
    # crop left part
    if left == 1:
        start_ = min(chrlen[chr_], start_ + lsize)
    
    # crop right part
    if right == 1:
        end_ = max(0, end_ - rsize)
        
    return (int(start_), int(end_))
    
def invert(conf):
    """
    Invert fragment with a probability conf[P_INVERT]
    """
    p = conf[P_INVERT]
    return random.choices([1,0], weights=[p,100-p], k=1)[0]
    
def duplicate(conf):
    """
    Duplicate fragment with a probability conf[P_DUP]
    """
    p = conf[P_DUP]
    return random.choices([1,0], weights=[p,100-p], k=1)[0]

def pick_nr_copies(conf):
    """
    Pick number of ecDNA copies
    """
    return random.choices(range(50,400, 10), k=1)[0]

def simulation(chrlen, chrranges, conf, type="random"):
    """
    Simulate a single ecDNA structure
    """
    
    ecDNA = []
    count_frag = -1
    while(len(ecDNA)<conf[N]):
        
        # 1. choose fragment
        chr_, start_, end_, len_ = choose_fragment(chrlen, chrranges, count_frag, ecDNA, conf, type=type)
        # print("Choose fragment Nr. chr start end len, ", count_frag, chr_, start_, end_, len_)
        
        # 2. emulate small deletions
        start_, end_ = small_deletions(chrlen, chr_, start_, end_, len_, conf)
        # print("Small del Nr. chr start end len, ", count_frag, chr_, start_, end_, len_)
        
        # 3. emulate inversion
        strand_ = invert(conf)
        count_frag += 1
        ecDNA.append((chr_,start_,end_,strand_,count_frag))
        
        # 4. allow segmental duplication        
        if count_frag < conf[N]:
            dup_ = duplicate(conf)
            if dup_ == 1:
                count_frag += 1
                ecDNA.append((chr_,start_,end_,strand_,count_frag))
                
    # 5. choose coverage
    cov_ = pick_nr_copies(conf)
                
    return ecDNA, cov_

def ecDNA2bed(ecDNAdict, fout):
    
    with open(fout, "w") as f:
        f.write("#chr\tstart\tstop\tdirection\ttarget\tcoverage\tstructure\tfragment\n")
        for circ in ecDNAdict:
            for i, frag in enumerate(ecDNAdict[circ]["structure"]):
                chr_,start_,end_,strand_,count_frag = frag
                strand_ = "+" if strand_ == 0 else "-"
                coverage_ = ecDNAdict[circ]["coverage"]
                circ_ = """circ_{}""".format(str(circ))
                f.write("""{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\n""".format(chr_,
                                                                     str(start_),
                                                                     str(end_),
                                                                     strand_,
                                                                     circ_,
                                                                     str(coverage_),
                                                                     circ_,
                                                                     str(i)
                                                                     ))
    
simulation(chrlen, chrranges, conf, type="random")

([('chr12', 107909794, 107930764, 0, 0),
  ('chr12', 78839978, 78858938, 0, 1),
  ('chr12', 78839978, 78858938, 0, 2)],
 330)

1. Simulate simple excision

conf = {N:1,
        NEIGHBOR:False,
        SINGLE_CHR:True,
        WEIGHT_CHR: [50,50],
        P_DEL_LEFT: 0,
        P_DEL_RIGHT: 0,
        P_INVERT: 0,
        P_DUP: 0
       }

In [98]:
# configuration for simple excisions
from collections import defaultdict
conf = {N:1,
        NEIGHBOR:True,
        SINGLE_CHR:True,
        WEIGHT_CHR: [50,50],
        P_DEL_LEFT: 0,
        P_DEL_RIGHT: 0,
        P_INVERT: 0,
        P_DUP: 0
       }
for i in range(0,5):
    ecDNAdict = defaultdict(int)
    for j in range(0,conf[N]):
        ecDNA, cov = simulation(chrlen, chrranges, conf, type="random")
        ecDNAdict[j]={}
        ecDNAdict[j]["structure"] = ecDNA
        ecDNAdict[j]["coverage"] = cov
    ecDNA2bed(ecDNAdict, """sim/excision/circ_1_{}.bed""".format(str(i)))

2. Simulate simple events (small deletions)

conf = {N:1,
        NEIGHBOR:True,
        SINGLE_CHR:True,
        WEIGHT_CHR: [50,50],
        P_DEL_LEFT: 60,
        P_DEL_RIGHT: 60,
        P_INVERT: 0,
        P_DUP: 0
       }

In [106]:
conf = {N:2,
        NEIGHBOR:True,
        SINGLE_CHR:True,
        WEIGHT_CHR: [50,50],
        P_DEL_LEFT: 60,
        P_DEL_RIGHT: 60,
        P_INVERT: 0,
        P_DUP: 0
       }

# define number of fragments
for k in range(2,10):
    conf[N] = k
    
    # simulation number
    for i in [1,2,3]:
        ecDNAdict = defaultdict(int)
        
        # how many ecDNA per file
        for j in range(0,1):
            ecDNA, cov = simulation(chrlen, chrranges, conf, type="nonrandom")
            ecDNAdict[j]={}
            ecDNAdict[j]["structure"] = ecDNA
            ecDNAdict[j]["coverage"] = cov
        ecDNA2bed(ecDNAdict, """sim/simple_del/circ_{}_{}.bed""".format(str(k),str(i)))

3. Simulate simple inversion 

conf = {N:1,
        NEIGHBOR:True,
        SINGLE_CHR:True,
        WEIGHT_CHR: [50,50],
        P_DEL_LEFT: 0,
        P_DEL_RIGHT: 0,
        P_INVERT: 40,
        P_DUP: 0
       }

In [107]:
conf = {N:2,
        NEIGHBOR:True,
        SINGLE_CHR:True,
        WEIGHT_CHR: [50,50],
        P_DEL_LEFT: 0,
        P_DEL_RIGHT: 0,
        P_INVERT: 40,
        P_DUP: 0
       }

# define number of fragments
for k in range(2,10):
    conf[N] = k
    
    # simulation number
    for i in [1,2,3]:
        ecDNAdict = defaultdict(int)
        
        # how many ecDNA per file
        for j in range(0,1):
            ecDNA, cov = simulation(chrlen, chrranges, conf, type="nonrandom")
            ecDNAdict[j]={}
            ecDNAdict[j]["structure"] = ecDNA
            ecDNAdict[j]["coverage"] = cov
        ecDNA2bed(ecDNAdict, """sim/simple_inv/circ_{}_{}.bed""".format(str(k),str(i)))

4. Simulate simple inversion and deletions

conf = {N:1,
        NEIGHBOR:True,
        SINGLE_CHR:True,
        WEIGHT_CHR: [50,50],
        P_DEL_LEFT: 60,
        P_DEL_RIGHT: 60,
        P_INVERT: 40,
        P_DUP: 0
       }

In [108]:
conf = {N:1,
        NEIGHBOR:True,
        SINGLE_CHR:True,
        WEIGHT_CHR: [50,50],
        P_DEL_LEFT: 60,
        P_DEL_RIGHT: 60,
        P_INVERT: 40,
        P_DUP: 0
       }

# define number of fragments
for k in range(2,10):
    conf[N] = k
    
    # simulation number
    for i in [1,2,3]:
        ecDNAdict = defaultdict(int)
        
        # how many ecDNA per file
        for j in range(0,1):
            ecDNA, cov = simulation(chrlen, chrranges, conf, type="nonrandom")
            ecDNAdict[j]={}
            ecDNAdict[j]["structure"] = ecDNA
            ecDNAdict[j]["coverage"] = cov
        ecDNA2bed(ecDNAdict, """sim/simple_mix/circ_{}_{}.bed""".format(str(k),str(i)))